# Titanic Dataset - Walkthrough with Ken Jee

Youtube link: https://www.youtube.com/watch?v=I3FBJdiExcg
Kaggle link: https://www.kaggle.com/code/kenjee/titanic-project-example/notebook

Following along with Ken Jee's tutorial on Kaggle.

## Step 1: Read the documentation


In [1]:
print('hello world')

hello world
